In [32]:
from selenium import webdriver
import selenium
from io import StringIO
# This is the path I use
# DRIVER_PATH = '.../Desktop/Scraping/chromedriver 2'
# Put the path for your ChromeDriver here
DRIVER_PATH = r"C:\Users\HP\Desktop\folders\others\chromedriver\chromedriver.exe"
wd = webdriver.Chrome(executable_path=DRIVER_PATH)

In [33]:
wd.get('https://google.com')

In [34]:
search_box = wd.find_element_by_css_selector('input.gLFyf')
search_box.send_keys('Dal Makhni images')

In [35]:
wd.quit()

In [36]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                print("something's wrong")
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))
#                     print(image_urls)

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [37]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [38]:
def search_and_download(search_term:str,driver_path:str,target_path='./images'):
    number_images = int(input('Enter the required number of images: '))
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=1.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [40]:
import io
import os
#from io import image
from PIL import Image
import requests
import hashlib
import time
# search_term='computers'

search_and_download(
    search_term=input('Enter the Image you want to download: '),
    driver_path=DRIVER_PATH
)
print(f"please check for 'images' folder to find the downloaded images in 'downloads' section")

Enter the Image you want to download: babe
Enter the required number of images: 1
Found: 100 search results. Extracting links from 0:100
Found: 1 image links, done!
SUCCESS - saved https://upload.wikimedia.org/wikipedia/en/b/b9/Babe_pig_in_the_city.jpg - as ./images\babe\e90abf501c.jpg
please check for 'images' folder to find the downloaded images in 'downloads' section
